In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# 📁 Paths
BASE_DIR = "C:\\Users\\MANJU\\Desktop\\FYP_Diabetes\\data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 🔢 Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 64
EPOCHS = 30

# 🧪 Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    shear_range=0.3,
    brightness_range=[0.6, 1.4],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# 📦 Data Loaders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# 📏 Class Weight Calculation
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_generator.classes),
                                     y=train_generator.classes)
class_weights = dict(enumerate(class_weights))

# 🧠 Load MobileNetV2 Base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze initially

# 🏗️ Custom Classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# 🧪 Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 📌 Callbacks
checkpoint = ModelCheckpoint("best_mobilenetv2_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# 🚀 Train Model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🧾 Evaluate Model
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"\n✅ Test Accuracy: {accuracy * 100:.2f}%")

# 💾 Save Final Model
model.save("final_mobilenetv2_model.h5")
print("✅ Fast and Accurate Model training complete!")


Found 112 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5134 - loss: 0.8744
Epoch 1: val_accuracy improved from -inf to 0.66667, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step - accuracy: 0.5179 - loss: 0.8622 - val_accuracy: 0.6667 - val_loss: 0.6410 - learning_rate: 3.0000e-04
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5372 - loss: 0.8043
Epoch 2: val_accuracy did not improve from 0.66667
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.5427 - loss: 0.8093 - val_accuracy: 0.5417 - val_loss: 0.6616 - learning_rate: 3.0000e-04
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5670 - loss: 0.7159
Epoch 3: val_accuracy did not improve from 0.66667
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.5893 - loss: 0.6878 - val_accuracy: 0.5000 - val_loss: 0.7939 - learning_rate: 3.0000e-04
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5804 - loss: 0.7090
Epoch 4: val_accuracy did not improve from 0.66667

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.5863 - loss: 0.7073 - val_accuracy: 0.5417 - val_loss: 0

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.7599 - loss: 0.5151 - val_accuracy: 0.7083 - val_loss: 0.5611 - learning_rate: 1.5000e-04
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7068 - loss: 0.5380
Epoch 7: val_accuracy improved from 0.70833 to 0.79167, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.7063 - loss: 0.5387 - val_accuracy: 0.7917 - val_loss: 0.4793 - learning_rate: 1.5000e-04
Epoch 8/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7604 - loss: 0.5676
Epoch 8: val_accuracy improved from 0.79167 to 0.87500, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.7569 - loss: 0.5624 - val_accuracy: 0.8750 - val_loss: 0.4446 - learning_rate: 1.5000e-04
Epoch 9/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7210 - loss: 0.5726
Epoch 9: val_accuracy did not improve from 0.87500
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.7217 - loss: 0.5682 - val_accuracy: 0.8750 - val_loss: 0.4314 - learning_rate: 1.5000e-04
Epoch 10/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7031 - loss: 0.5426
Epoch 10: val_accuracy improved from 0.87500 to 0.91667, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6979 - loss: 0.5400 - val_accuracy: 0.9167 - val_loss: 0.4244 - learning_rate: 1.5000e-04
Epoch 11/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7485 - loss: 0.5765
Epoch 11: val_accuracy did not improve from 0.91667
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.7550 - loss: 0.5569 - val_accuracy: 0.8750 - val_loss: 0.4283 - learning_rate: 1.5000e-04
Epoch 12/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7600 - loss: 0.5043
Epoch 12: val_accuracy did not improve from 0.91667
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.7478 - loss: 0.5249 - val_accuracy: 0.7917 - val_loss: 0.4417 - learning_rate: 1.5000e-04
Epoch 13/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7411 - loss: 0.5010
Epoch 13: val_accuracy did not improve from 0.91667

Epoch 13: ReduceLROnPlateau reducing learning rate to 7.500000356230885e-05.
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.7589 - loss: 0.4784 - val_accuracy: 0.7917 - val_l


✅ Test Accuracy: 75.00%
✅ Fast and Accurate Model training complete!
